In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time
import numpy as np
import pandas as pd
import pickle
import os

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import warnings
warnings.filterwarnings("ignore")

In [3]:
with open('./config.json', 'r') as fo:
    conf = json.loads(fo.read())
kib_login = conf['kib_login']
kib_password = conf['kib_password']
data_folder = './parsed_data'

# Пробуем парсинг К&Б через selenium

In [4]:
from src.kib_scrapper import LocalCatalogItems, KibCatalog, ShopSelector

In [5]:
driver = webdriver.Safari()

In [6]:
catalog = KibCatalog(driver, kib_login=kib_login, kib_password=kib_password, save_path=data_folder)

In [7]:
cities = []
for macro, d in catalog.shop_list.items():
    cities += list(d.keys())

cities_need = """КАЗАНЬ
САМАРА
ЯРОСЛАВЛЬ
НИЖНИЙ НОВГОРОД
МОСКВА
КРАСНОДАР
САНКТ-ПЕТЕРБУРГ
НОВОСИБИРСК
Екатеринбург
Пермь
ВОРОНЕЖ
ТУЛА
Ростов-на-Дону
Челябинск"""
cities_need = [c.lower() for c in cities_need.split('\n')]
regions_to_parse = []
for city in cities_need:
    for macro, d in catalog.shop_list.items():
        found = False
        for c in d.keys():
            if city in c.lower():
                found = True
                regions_to_parse.append((macro, c))
                break
        if found:
            break
    else:
        print(f'Не получилось найти город {city}')

adresses = []
for macro, city in regions_to_parse:
    adresses.append((macro, city, np.random.choice(catalog.shop_list[macro][city])))

In [7]:
adresses = [
    ('Республика Татарстан', 'Казань', 'ул. Маршала Чуйкова, 63а'),
    ('Самарская область', 'Самара', 'ул. Советской Армии, 17'),
    ('Ярославская область', 'Ярославль', 'пр-кт Ленинградский, 66а'),
    ('Нижегородская область', 'Нижний Новгород', 'ул. Василия Иванова, 9'),
    ('Москва', 'Москва', 'ул. Весенняя, 4'),
    ('Краснодарский край', 'Краснодар', 'ул. Агрохимическая, 117/1'),
    ('Санкт-Петербург', 'Санкт-Петербург', 'пр-т Пискаревский, 10а'),
    ('Новосибирская область', 'Новосибирск', 'ул. Комсомольская, 2/2'),
    ('Свердловская область', 'Екатеринбург', 'ул. Первомайская, 35'),
    ('Пермский край', 'Пермь', 'ул. Уинская, 35'),
    ('Воронежская область', 'Воронеж', 'ул. Березовая роща, 62б'),
    ('Тульская область', 'Тула', 'ул. Кутузова, 43'),
    ('Ростовская область', 'Ростов-на-Дону', 'б-р Комарова, 20'),
    ('Челябинская область', 'Челябинск', 'ул. Доватора, 42')
]

**Парсинг всего каталога**

In [8]:
# %%time
# # catalog = KibCatalog(driver, ['Москва', 'Курская область'], ['Вода, соки, напитки'])
# catalog = KibCatalog(driver, kib_login=kib_login, kib_password=kib_password, save_path=data_folder, regions = )
# # with open('all_kib_catalog.pickle', 'wb') as fo:
# #     pickle.dump(catalog.catalogs, fo)
# # catalog.parse_all_catalogs(['Бакалея, чай, кофе'])
# for i in range(10):
#     try:
#         catalog.parse_all_catalogs()
#     except:
#         driver.quit()
#         driver = webdriver.Safari()
#         catalog = KibCatalog(driver)

**Парсинг отдельных магазинов**

In [9]:
categories = [
    'Водка, настойки', 'Виски, бурбон', 'Коньяк, арманьяк',
    'Текила, ром, ликер', 'Вода, соки, напитки', 'Мясо, рыба, икра',
    'Молоко, сыры, яйцо', 'Бакалея, чай, кофе', 'Консервация, салаты',
    'Шоколад, конфеты, мороженое', 'Хлеб, выпечка, печенье', 'Снэки',
    'Пиво Россия', 'Корм для животных'
]

In [10]:
# adresses = [
# #     ('Москва', 'Москва', 'пер. Лялин, 22'),
# #     ('Москва', 'Москва', 'б-р Академика Ландау, 3'),
#     ('Москва', 'Москва', 'Михайловский Верхн. 2-й проезд, 6')
# ]
for ad in adresses:
    tag = '_'.join(ad).replace('/', '-')
    filename = 'kib_' + tag + '.csv'
    print(filename)
    if filename in os.listdir('./parsed_data'):
        print('already done')
        continue
    
    file_path = './parsed_data/' + filename
    items = catalog.parse_catalog(*ad, categories_needed=categories)
    df = items.get_dataframe()
    
    df.to_csv(file_path, index=False)
    driver.get('https://krasnoeibeloe.ru')
    time.sleep(10)

kib_Республика Татарстан_Казань_ул. Маршала Чуйкова, 63а.csv
already done
kib_Самарская область_Самара_ул. Советской Армии, 17.csv
already done
kib_Ярославская область_Ярославль_пр-кт Ленинградский, 66а.csv
already done
kib_Нижегородская область_Нижний Новгород_ул. Василия Иванова, 9.csv
already done
kib_Москва_Москва_ул. Весенняя, 4.csv
already done
kib_Краснодарский край_Краснодар_ул. Агрохимическая, 117-1.csv
already done
kib_Санкт-Петербург_Санкт-Петербург_пр-т Пискаревский, 10а.csv
already done
kib_Новосибирская область_Новосибирск_ул. Комсомольская, 2-2.csv
already done
kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv
already done
kib_Пермский край_Пермь_ул. Уинская, 35.csv
already done
kib_Воронежская область_Воронеж_ул. Березовая роща, 62б.csv
parsing Водка, настойки
parsing Виски, бурбон
parsing Коньяк, арманьяк
parsing Текила, ром, ликер
parsing Вода, соки, напитки
parsing Мясо, рыба, икра
parsing Молоко, сыры, яйцо
parsing Бакалея, чай, кофе
parsing Консервация

# Собираем файл для ребят

In [11]:
files = os.listdir('./parsed_data')
df = pd.read_csv('./parsed_data/kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv')
df = df[df.region.isnull()]
for file in files:
    _, ext = os.path.splitext(file)
    if ext != '.csv':
        continue
    print(file)
    curr_df = pd.read_csv('./parsed_data/' + file)
    df = pd.concat([df, curr_df])

kib_Республика Татарстан_Казань_ул. Маршала Чуйкова, 63а.csv
kib_Самарская область_Самара_ул. Советской Армии, 17.csv
kib_Краснодарский край_Краснодар_ул. Агрохимическая, 117-1.csv
kib_Пермский край_Пермь_ул. Уинская, 35.csv
kib_Санкт-Петербург_Санкт-Петербург_пр-т Пискаревский, 10а.csv
kib_Воронежская область_Воронеж_ул. Березовая роща, 62б.csv
kib_Нижегородская область_Нижний Новгород_ул. Василия Иванова, 9.csv
kib_Ростовская область_Ростов-на-Дону_б-р Комарова, 20.csv
kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv
kib_Ярославская область_Ярославль_пр-кт Ленинградский, 66а.csv
kib_Москва_Москва_ул. Весенняя, 4.csv
kib_Челябинская область_Челябинск_ул. Доватора, 42.csv
kib_Тульская область_Тула_ул. Кутузова, 43.csv
kib_Новосибирская область_Новосибирск_ул. Комсомольская, 2-2.csv


In [12]:
# Выбор категорий для парсинга
# matching = pd.read_excel('./data/matching.xlsx')
# print(matching.shape)
# matching = matching[['name', 'PLU ТС5']].drop_duplicates()
# matching.columns = ['name', 'plu_ts5']
# print(matching.shape)
# df.merge(matching, on=['name'], how='inner')['category'].unique()

In [13]:
# Преобразование цены во float
df['price'] = df.price.str.extract(r'(\d*\.\d*)')
df['price'] = df['price'].astype(float)
df.dropna(subset=['price'], inplace=True)

# Создание таблицы матчинга
# matching = pd.read_excel('./data/Book1.xlsx')[['id', 'name', 'PLU ТС5']]
# matching.columns = ['id', 'name', 'plu_ts5']
# matching.dropna(subset=['plu_ts5'], inplace=True)
# matching['plu_ts5'] = matching['plu_ts5'].astype(int).astype(str)
# matching = matching.drop_duplicates()
# matching.to_excel('./data/matching.xlsx', index=False)

# Применение матчинга
matching = pd.read_excel('./data/matching.xlsx')[['name', 'PLU ТС5']].drop_duplicates()
matching.columns = ['name', 'plu_ts5']
matching['plu_ts5'] = matching['plu_ts5'].astype(str)
df = df.merge(matching, on='name', how='left')

# сбор общей статистики
sumup = df.dropna(subset=['plu_ts5']).groupby('plu_ts5')['price'].mean().reset_index()

In [14]:
# Сохранение данных
writer = pd.ExcelWriter('parsed_data/kib_parsing_2022_02_06.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = 'date', index=False)
sumup.to_excel(writer, sheet_name = 'sumup', index=False)
writer.save()
writer.close()

In [7]:
# df.to_csv('parsed_data/kib_parsing_2022_01_30.csv', index=False, encoding='utf-16')

In [5]:
df.to_excel('parsed_data/kib_parsing_2022_01_30.xlsx', index=False)